<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Options_Trading_Strategy_based_on_ARIMA_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ibapi

from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import *
from ibapi.wrapper import EWrapper

class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = 0
        self.data = {}  # Store option data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def contractDetails(self, reqId: int, contractDetails):
        super().contractDetails(reqId, contractDetails)
        contract = contractDetails.contract
        symbol = contract.symbol
        right = contract.right
        self.data[symbol] = {"right": right, "strike": contract.strike}

    def tickPrice(self, reqId, tickType, price, attrib):
        super().tickPrice(reqId, tickType, price, attrib)
        if tickType == 1:  # Last price
            symbol = self.data[reqId]["symbol"]
            right = self.data[reqId]["right"]
            strike = self.data[reqId]["strike"]
            print(f"Received last price for {symbol}: {price}")
            # Implement your buy order logic here
            if price > threshold:
                self.placeOrder(symbol, right, strike, "BUY", quantity)
            # Implement your sell order logic here
            if price < threshold:
                self.placeOrder(symbol, right, strike, "SELL", quantity)

    def placeOrder(self, symbol, right, strike, action, quantity):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "20231215"
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"

        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = limit_price

        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1

app = TradingApp()
app.connect("127.0.0.1", 4002, clientId=1)

# Define your options contract details
symbols = ["AAPL", "TSLA", "GOOGL"]
quantity = 1
threshold = 1.0  # Define your threshold value
limit_price = 0.0  # Define your limit price

# Request market data for the options contracts
for symbol in symbols:
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = "20231215"
    contract.strike = 150
    contract.right = "C"
    contract.multiplier = "100"

    app.reqMarketDataType(4)
    app.reqContractDetails(len(app.data), contract)
    app.reqMktData(len(app.data), contract, "", False, False, [])
    app.nextOrderId += 1

app.run()


Title: Options Trading Strategy based on ARIMA Forecasting

Description:
This notebook implements an options trading strategy based on ARIMA (AutoRegressive Integrated Moving Average) forecasting. The strategy is inspired by the research article titled "Options trading strategy based on ARIMA forecasting" by Pierre Rostan, Alexandra Rostan, and Mohammad Nurunnabi. The article was published in the Department of Business, American University of Afghanistan, and the Department of Accounting, Prince Sultan University.

The strategy utilizes ARIMA models to forecast the future price movements of specific assets, such as stocks or indices, and makes trading decisions based on the forecasted values. By integrating ARIMA forecasting into options trading, the strategy aims to identify potential opportunities for profit and manage risk effectively.

Note: The implementation in this notebook is based on the concepts presented in the mentioned research article and may have been adapted to fit the Interactive Brokers trading platform.
REF: Rostan, P., Rostan, A., & Nurunnabi, M. (Year). Options trading strategy based on ARIMA forecasting. Journal Name, Volume(Issue), Page numbers.